# Interconnected Graph Builder

Here you can find all processes of building our dataset - graph of Innopolis city. All coordinates of city buildings were collected manualy with use of 'Yandex Maps'. Now all we need to complete our dataset is to build graph.

In [22]:
import os
import json
from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

Some functions for retriving and saving data

In [2]:
def get_data(path: str) -> list:
    """ Extracts nodes and graph information from given json file

    Args:
        path (str): Path to file

    Returns:
        List of nodes with its names, ids, and coords
    """
    PATH = os.path.abspath(path)

    with open(PATH, 'r') as file:
        data = json.load(file)
        nodes = data['nodes']

        return nodes

In [3]:
def save_data(path: str, nodes: list, graph: list) -> None:
    """ Saves nodes and graph to json file

    Args:
        path (str): Path to json
        nodes (list): List of nodes
        graph (list): 2D array of distances
    """
    PATH = os.path.abspath(path)

    data = {
        'nodes': nodes,
        'graph': graph
    }

    with open(PATH, 'w') as file:
        json.dump(data, file)

Additionally we add functionality to plot graphs

In [50]:
def plot_graph(nodes: list, paths:list = None) -> None:
    """ Plots graph and paths if needed

    Args:
        nodes (list): List of nodes
        paths (list): List of paths 
    """
    
    df = pd.DataFrame(nodes)

    fig = px.scatter_map(df,
                        lat="atd",
                        lon="lng",
                        hover_name="name",
                        color="id",
                        color_continuous_scale=px.colors.sequential.Viridis,
                        zoom=14,
                        height=800,
                        width=800)
    
    if paths is not None:
        coord_map = {row['id']: (row['atd'], row['lng']) for _, row in df.iterrows()}
        
        path_colors = px.colors.qualitative.Plotly
        
        for i, path in enumerate(paths):
            lats, lons = [], []
            for node_id in path:
                lat, lon = coord_map[node_id]
                lats.append(lat)
                lons.append(lon)
            
            fig.add_trace(go.Scattermap(
                mode="lines+markers",
                lon=lons,
                lat=lats,
                line=dict(width=3, color=path_colors[i % len(path_colors)]),
                name=f"Path {i+1}",
                marker=dict(size=8, color=path_colors[i % len(path_colors)]),
                hoverinfo="none"
            ))

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
                    coloraxis_showscale=False)
    fig.show()

To convert longitude and attitude to distances in graph **geopy** library were used 

In [4]:
def get_distances(nodes: list) -> list:
    """ Generates 2D array of distances

    Args:
        nodes (list): List of nodes

    Returns:
        list: 2D array of distances
    """
    graph = [[0 for _ in range(len(nodes))] for __ in range(len(nodes))]

    for node1 in nodes:
        for node2 in nodes:
            if node1['id'] == node2['id']:
                continue

            cord1 = (node1['atd'], node1['lng'])
            cord2 = (node2['atd'], node2['lng'])

            graph[node1['id']][node2['id']] = geodesic(cord1, cord2).m
            graph[node2['id']][node1['id']]  = graph[node1['id']][node2['id']] 

    return graph

Now we can convert coordinates to distances and save them in **graph.json** file

In [51]:
nodes = get_data('../data/nodes.json')
graph = get_distances(nodes)
save_data('../data/graph.json', nodes, graph)

Now we can plot our graph and try to plot some paths

In [52]:
paths = [[1, 2, 3, 4],
         [12, 10, 5, 6],
         [31, 32, 33],
         [22, 21]]

plot_graph(nodes, paths)